# Boyer Romain - Dechorgnat Paul - Kallala Inès - Ouahes Iza
# Kaggle - Titanic
## I - Libraries & Files

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from operator import itemgetter

In [2]:
path = r"C:\Users\Romain\Documents\230 - MSc ESSEC\304 - Machine Learning\02 - Kaggle Titanic"

train = pd.read_csv(path+'/Datasets/train.csv')
test_users = pd.read_csv(path+r"\Datasets\test.csv")
submission = pd.read_csv(path+'/Datasets/gender_submission.csv')

IOError: File C:\Users\Romain\Documents\230 - MSc ESSEC\304 - Machine Learning\02 - Kaggle Titanic/Datasets/train.csv does not exist

## II - Data preprocessing & feature engineering
### 1 - Functions
#### Creating a crosstab function to plot a graph

In [ ]:
def graph_crosstab(data = train, feature = 'Age', target_feature = 'Survived', normalize = False, title=1):
    mat=pd.crosstab(data[feature], data[target_feature], normalize=normalize)
    mat.plot(kind = 'bar', figsize=(13.5/1.5, 6/1.5))
    if title==1:
        plt.title(feature)
    else:
        plt.title(title)
    plt.show()
    return

#### Creating function to transform string values into numerical values

In [ ]:
def from_str_to_int(data, cat_index, cat, cat_dico, new_cat=''):
    if new_cat!='':
        for i in range(len(cat_index)):
            data.loc[data[cat]==cat_index[i], new_cat]=i
            cat_dico[cat_index[i]]=i
        cat_dico['Other']=len(cat_index)
    else:
        for i in range(len(cat_index)):
            data.loc[data[cat]==cat_index[i], cat]=i
            cat_dico[cat_index[i]]=i
    return

### 2 - First approach
#### Plot different graphs to get a first idea of the importance of a feature in survivability

In [ ]:
graph_crosstab(feature='Sex')
graph_crosstab(feature='Pclass')
graph_crosstab(feature='SibSp', normalize=0, title='Sibling / Spouse')
graph_crosstab(feature='Parch', normalize=0, title='Parents / Children')

#### Link between columns 'Pclass' & 'Fare'
#### Find the outliers in 'Fare'

In [ ]:
PClass_palette = {1:"b", 2:"y", 3:"r"}
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
sns.boxplot( x=train["Pclass"], y=train["Fare"], palette=PClass_palette, showfliers=True)
plt.title('With outliers')

plt.subplot(1,2,2)
sns.boxplot( x=train["Pclass"], y=train["Fare"], palette=PClass_palette, showfliers=False)
plt.title('Without outliers')
plt.show()

In [ ]:
PClass_palette = {1:"b", 2:"y", 3:"r"}
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
sns.boxplot( x=test_users["Pclass"], y=test_users["Fare"], palette=PClass_palette, showfliers=True)
plt.title('With outliers')

plt.subplot(1,2,2)
sns.boxplot( x=test_users["Pclass"], y=test_users["Fare"], palette=PClass_palette, showfliers=False)
plt.title('Without outliers')
plt.show()

### 3 - Create new columns
#### Create new 'Title' & 'Last_Name' columns from 'Name' column

In [ ]:
def create_col_titre_name(data=train):
    data['Titre'] = data['Name'].apply(lambda x : str(x).split(',')[1].split('.')[0][1:])
    data['Titre'] = data['Titre'].astype('category')
    data['Last_Name'] = data['Name'].apply(lambda x : str(x).split(',')[0].replace(' ',''))
    return

create_col_titre_name(train)
create_col_titre_name(test_users)

In [ ]:
graph_crosstab(feature='Titre', normalize=True)

#### Create 5 groups with 'Title'

In [ ]:
titre_index=['Mr','Miss','Mrs', 'Master']

train['Titre_int']=int(len(titre_index))
titre_dico=dict()
from_str_to_int(data=train, cat_index=titre_index, cat='Titre', cat_dico=titre_dico, new_cat='Titre_int')

test_users['Titre_int']=int(len(titre_index))
titre_dico_test=dict()
from_str_to_int(data=test_users, cat_index=titre_index, cat='Titre', cat_dico=titre_dico_test, new_cat='Titre_int')

In [ ]:
x=[0,1,2,3,4]
labels=['Mr','Miss','Mrs','Master','Other']

pd.crosstab(train['Titre_int'], train['Survived'], normalize=False).plot(kind='bar', figsize=(14,6))

plt.xticks(x, labels, rotation=45)

plt.show()

#### Create 'Family' column and 3 new groups (no family on board; 1,2 or 3 family members on board; more than 3 family member on board)

In [ ]:
train['Family']=train['SibSp']+train['Parch']
test_users['Family']=test_users['SibSp']+test_users['Parch']

In [ ]:
graph_crosstab(feature='Family', normalize=0)

In [ ]:
train['Family_group']=int(2)
train.loc[train['Family']==0, 'Family_group']=0
train.loc[train['Family']==1, 'Family_group']=1
train.loc[train['Family']==2, 'Family_group']=1
train.loc[train['Family']==3, 'Family_group']=1

test_users['Family_group']=int(2)
test_users.loc[test_users['Family']==0, 'Family_group']=0
test_users.loc[test_users['Family']==1, 'Family_group']=1
test_users.loc[test_users['Family']==2, 'Family_group']=1
test_users.loc[test_users['Family']==3, 'Family_group']=1

In [ ]:
x=[0,1,2]
labels=['Alone','1,2 or 3 family members on board','More than 3 family members on board']

pd.crosstab(train['Family_group'], train['Survived'], normalize=False).plot(kind='bar', figsize=(14,6))

plt.xticks(x, labels, rotation=0)

plt.show()

#### Create 'Cabin_lettre' column, with the first letter of the cabin, or 'Z' if the cabin is not filled

In [ ]:
train['Cabin_lettre'] = train['Cabin'].fillna('Z').apply(lambda x : str(x)[0])
test_users['Cabin_lettre'] = test_users['Cabin'].fillna('Z').apply(lambda x : str(x)[0])

In [ ]:
graph_crosstab(feature='Cabin_lettre', normalize=True)

In [ ]:
train['Cabin_lettre_group']=int(2)
train.loc[train['Cabin_lettre']=='Z', 'Cabin_lettre_group']=0
train.loc[train['Cabin_lettre']=='A', 'Cabin_lettre_group']=1
train.loc[train['Cabin_lettre']=='G', 'Cabin_lettre_group']=1

test_users['Cabin_lettre_group']=int(2)
test_users.loc[train['Cabin_lettre']=='Z', 'Cabin_lettre_group']=0
test_users.loc[train['Cabin_lettre']=='A', 'Cabin_lettre_group']=1
test_users.loc[train['Cabin_lettre']=='G', 'Cabin_lettre_group']=1

In [ ]:
x=[0,1,2]
labels=['Cabin not filled','Cabins A or G','Cabins B, C, D, E, F or T']

pd.crosstab(train['Cabin_lettre_group'], train['Survived'], normalize=False).plot(kind='bar', figsize=(14,6))

plt.xticks(x, labels, rotation=0)

plt.show()

#### Create 'Ticket_lettre' column with the first letter of the ticket

In [ ]:
train['Ticket_lettre'] = train['Ticket'].apply(lambda x : str(x)[0])
test_users['Ticket_lettre'] = test_users['Ticket'].apply(lambda x : str(x)[0])

In [ ]:
graph_crosstab(feature='Ticket_lettre', normalize=False)

### 4 - Fill train / test
#### Fill 'Age' with the mean of ages by titles

In [ ]:
def fill_age_with_titre(data):
    cat_titre=list((train['Titre'].cat.categories))
    for i in cat_titre:
        data.loc[data['Titre']==i, 'Age']=data.loc[data['Titre']==i, 'Age'].fillna(int(np.mean(train['Age'][train['Titre']==i])))
    return

fill_age_with_titre(train)
fill_age_with_titre(test_users)

#### Fill 'Embarked' with the same harbour than a family member, or with the most representative harbour 'S'

In [ ]:
train['Embarked']=train['Embarked'].fillna('S')

#### Fill 'Fare' with the mean of 'Pclass'

In [ ]:
test_users.loc[(test_users['Fare'].isnull())&(test_users['Pclass']==3), 'Fare']=int(np.mean(test_users['Fare'][test_users['Pclass']==3]))

### 5 - Transform columns into dummy ones, to avoid intern classification
#### Transform string col Sex into binary one

In [ ]:
sex_index = ['male', 'female']

sex_dico=dict()
from_str_to_int(data=train, cat_index=sex_index, cat='Sex', cat_dico=sex_dico)

sex_dico_test=dict()
from_str_to_int(data=test_users, cat_index=sex_index, cat='Sex', cat_dico=sex_dico_test)

In [ ]:
# Dummy variables pour les Embarked
train = pd.get_dummies(train, columns=['Embarked'])
test_users = pd.get_dummies(test_users, columns=['Embarked'])

# Dummy variables for Titre_int
train = pd.get_dummies(train, columns=['Titre_int'])
test_users = pd.get_dummies(test_users, columns=['Titre_int'])

# Dummy variables for Family_group
train = pd.get_dummies(train, columns=['Family_group'])
test_users = pd.get_dummies(test_users, columns=['Family_group'])

# Dummy variables for Cabin_lettre
train = pd.get_dummies(train, columns=['Cabin_lettre'])
test_users = pd.get_dummies(test_users, columns=['Cabin_lettre'])

# Dummy variables pour Cabin_lettre_group
train = pd.get_dummies(train, columns=['Cabin_lettre_group'])
test_users = pd.get_dummies(test_users, columns=['Cabin_lettre_group'])

# Dummy variables pour Ticket_lettre
train = pd.get_dummies(train, columns=['Ticket_lettre'])
test_users = pd.get_dummies(test_users, columns=['Ticket_lettre'])

## III - Learning algorithm
### 1 - Cross Validation on different models

In [ ]:
features_of_interest_CrossVal=["Pclass", "Parch", "SibSp", "Sex", "Age", "Fare",
                         'Titre_int_0', "Titre_int_1", "Titre_int_2", "Titre_int_3", "Titre_int_4",
                         "Embarked_C", "Embarked_Q", "Embarked_S",
                         'Cabin_lettre_A', 'Cabin_lettre_B', 'Cabin_lettre_C', 'Cabin_lettre_D', 
                         'Cabin_lettre_E', 'Cabin_lettre_F', 'Cabin_lettre_G', 'Cabin_lettre_Z',
                         'Cabin_lettre_group_0', 'Cabin_lettre_group_1', 'Cabin_lettre_group_2',
                         'Family_group_0', 'Family_group_1', 'Family_group_2',
                         'Ticket_lettre_1', 'Ticket_lettre_2', 'Ticket_lettre_3', 'Ticket_lettre_4', 
                         'Ticket_lettre_6', 'Ticket_lettre_7', 'Ticket_lettre_9', 'Ticket_lettre_A', 
                         'Ticket_lettre_C', 'Ticket_lettre_F', 'Ticket_lettre_L', 'Ticket_lettre_P', 
                         'Ticket_lettre_S', 'Ticket_lettre_W']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KDTree

In [ ]:
X_train = train[features_of_interest_CrossVal]
X_Survived = train['Survived']

def fun_cross_val_score(my_MODEL, algo, cv=20):
    scores = cross_val_score(my_MODEL, X_train, X_Survived, cv=cv)
    return print("Accuracy %s: %0.4f (+/- %0.2f)" % (algo ,scores.mean(), scores.std() * 2))

In [ ]:
models=dict()
models['Random Forest 1']=RandomForestClassifier()

models['Random Forest 2'] = RandomForestClassifier(
    n_estimators=21, min_samples_split=10, max_leaf_nodes=10, max_depth=10, random_state=0)

models['Random Forest 3'] = RandomForestClassifier(
    n_estimators=20, min_samples_split=10, max_leaf_nodes=12, max_depth=12, random_state=0)

models['Logistic Regression 1'] = LogisticRegression()

models['Ada Boost 1'] = AdaBoostClassifier()

models['Gradient Boosting Classifier'] = GradientBoostingClassifier()

models['Decision Tree Classifier'] = DecisionTreeClassifier()

In [ ]:
for key, value in models.items():
    fun_cross_val_score(value, key)

In [ ]:
list_test=list()
for k,v in models.items():
    list_test.append((k,v))

In [ ]:
fun_cross_val_score(VotingClassifier(estimators=list_test, voting='hard'), 'voting')

### 2 - Optimization of these models

In [ ]:
from sklearn.model_selection import GridSearchCV

### 3 - Random Forest (0.81339 on Kaggle)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
target_RF = train['Survived'].values

features_of_interest_RF=features_of_interest_CrossVal

features_RF = train[features_of_interest_RF].values

my_RF = RandomForestClassifier(n_estimators=21, min_samples_split=10, max_leaf_nodes=10, max_depth=10, random_state=0)

my_RF = my_RF.fit(features_RF, target_RF)

print(my_RF.score(features_RF, target_RF))

feature_of_importance = (sorted(zip(map(lambda x: round(int(x*10000)/100, 15), my_RF.feature_importances_), features_of_interest_RF), 
             reverse=False))

features=pd.DataFrame()
for i in range(len(feature_of_importance)):
    features.loc[i, 'Feature']=feature_of_importance[i][1]
    features.loc[i, 'Score']=feature_of_importance[i][0]
    
features.set_index('Feature', inplace=True)
features.plot(kind='barh', figsize=(9,9), title='Features importance')

test_features_RF = test_users[features_of_interest_RF].values

my_prediction_RF = my_RF.predict(test_features_RF)

PassengerId =np.array(test_users["PassengerId"]).astype(int)
my_solution_RF = pd.DataFrame(my_prediction_RF, PassengerId, columns = ["Survived"])

my_solution_RF.to_csv("my_solution_RF.csv", index_label = ["PassengerId"])

### 3 - LogisticRegression (0.77511 on Kaggle)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
target_LR = train['Survived'].values

features_of_interest_LR=features_of_interest_CrossVal

features_LR = train[features_of_interest_LR].values

my_LR = LogisticRegression(random_state=0)
my_LR = my_LR.fit(features_LR, target_LR)

print(my_LR.score(features_LR, target_LR))

feature_of_importance = (sorted(zip(map(lambda x: (round(int(x*10000)/100, 15)), my_LR.coef_[0]), features_of_interest_LR), 
             reverse=False))

features=pd.DataFrame()
for i in range(len(feature_of_importance)):
    features.loc[i, 'Feature']=feature_of_importance[i][1]
    features.loc[i, 'Score']=feature_of_importance[i][0]
    
features.set_index('Feature', inplace=True)
features.plot(kind='barh', figsize=(9,9), title='Features importance')

test_features_LR = test_users[features_of_interest_LR].values

my_prediction_LR = my_LR.predict(test_features_LR)

PassengerId =np.array(test_users["PassengerId"]).astype(int)
my_solution_LR = pd.DataFrame(my_prediction_LR, PassengerId, columns = ["Survived"])

my_solution_LR.to_csv("my_solution_LR.csv", index_label = ["PassengerId"])

### End - VotingClassifier (0.81818 on Kaggle)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

target_VC = train['Survived'].values

features_of_interest_VC=features_of_interest_CrossVal


features_VC = train[features_of_interest_VC].values

clf1 = LogisticRegression(random_state=0)
clf2 = RandomForestClassifier(n_estimators=21, min_samples_split=10, max_leaf_nodes=10, max_depth=10, random_state=0)


eclf = VotingClassifier(estimators=[('lr', clf1), 
                                    ('rf', clf2)], voting='hard')

my_VC = eclf.fit(features_VC, target_VC)

for clf, label in zip([clf1, 
                       clf2,
                       eclf], ['Logistic Regression', 'Random Forest', 'Ensemble']):
    scores = cross_val_score(clf, features_VC, target_VC, cv=15, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))
    
print(my_VC.score(features_VC, target_VC))

test_features_VC = test_users[features_of_interest_VC].values

my_prediction_VC = my_VC.predict(test_features_VC)

PassengerId =np.array(test_users["PassengerId"]).astype(int)
my_solution_VC = pd.DataFrame(my_prediction_VC, PassengerId, columns = ["Survived"])

my_solution_VC.to_csv("my_solution_VC.csv", index_label = ["PassengerId"])